In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import ipywidgets as w
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import numpy as np

In [15]:
import ez_torch as ez
from ez_torch.ui import make
import torch.nn as nn
from ez_torch.modules import Module, SpatialUVOffsetTransformer
import torch

from threading import Thread

In [16]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.net(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    
    def training_step(self, batch):
        optim = self.optimizers()

        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        return loss.item()

In [17]:
train, test = ez.data.get_mnist_dl(bs_train=32, bs_test=32, shuffle=True)

In [18]:
model = Model()

In [19]:
ez.ui.for_training(model, dataloader=train, in_background=True, its=1000)

In [20]:
from ez_torch.ui import make as ui
view = ui.v(
    ui.checkbox('running', description='Stop'),
    ui.output('area')
)

In [21]:
import time

In [22]:
def run():
    i = 0
    while view.running.value:
        time.sleep(0.01)
        i += 1
        with view.area:
            print(i)

t = Thread(target=run)
t.start()

view

In [23]:
from ez_torch.ui import make as ui

In [24]:
view = ui.v(
    ui.h(
        ui.v(
            ui.label(value='LR'),
            ui.slider('lr', min=0.001, max=1, layout={'width': 'auto'}),
            ui.label(value='ITS'),
            ui.int('its', min=1, max=1000, default=100, layout={'width': 'auto'}),
            layout=w.Layout(**{'width': '25%', 'border': '2px solid #ccc'}),
        ),
        ui.v(
            ui.h(
                ui.progress('progress', description='IT [000:000]'),
                ui.button('start', description='Start'),
                ui.button('reset', description='Restart'),
            ),
            ui.line('loss', title='Loss', layout={'height': '350px'}),
            layout={'width': '100%', 'border': '2px solid #ccc'},
        ),
        layout={'border': '2px solid #ccc'},
    ),
    ui.output('out'),
    layout={'border': '4px solid #e6e6e6', 'paddnig': '10px'},
)

view

In [25]:
row = ui.h(ui.output('left'), ui.output('right'))
view.out.display(row)

In [26]:
import threading

def run():
    n = 10
    y = np.random.rand(n)
    for i in range(1, n):

        view.loss.plot(y[:i])
        with row.left:
            row.left.clear()
            plt.imshow(np.random.rand(10, 10))
            plt.show()
        with row.right:
            row.left.clear()
            plt.imshow(np.random.rand(10, 10))
            plt.show()

In [28]:
# t = threading.Thread(target=run)
# t.start()
run()

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ez_torch.ui as ui
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
def experiment(display):
    for i in np.linspace(0, 1, 100):
        yield i
        with display:
#             display.clear()
            A = np.random.rand(10, 10)
            plt.imshow(A)
            plt.show(block=True)

In [4]:
view = ui.for_generator(experiment)
view

In [9]:
from IPython.display import display, clear_output
with view.out:
    clear_output(wait=True)
    display()

In [10]:
view.out

Output()